In [81]:
import pandas as pd
import numpy as np
from threading import Thread, Lock
from os.path import exists
%run NeuralNetGenerator.ipynb
np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

In [82]:
class CriticTrainingFunctions:
    writeLock = Lock()
    stateFile = "critic_states.csv"
    def formatInitialCsv():
        if not exists(CriticTrainingFunctions.stateFile):
            with open(CriticTrainingFunctions.stateFile, "w") as crtcFile:
                strToWrite = ','.join(str(i) for i in range(SEQUENTIALINPUTSHAPE[0]))
                strToWrite += (',critic,won' + '\n')
                crtcFile.write(strToWrite)
    def writeStates(stateArray, returns, won, size):
        returnList = returns.numpy()
        with CriticTrainingFunctions.writeLock:
            with open(CriticTrainingFunctions.stateFile, "a") as states:
                 for i in range(size):
                    lst = stateArray.read(i).numpy().tolist()
                    lst = [str(i) for i in lst]
                    lstStr = ",".join(lst) + "," + str(returnList[i]) + "," + str(1 if won else -1) + "\n"
                    states.write(lstStr)
    def getFileSize():
        with open(CriticTrainingFunctions.stateFile, "r") as crtcFile:
            return len(crtcFile.readlines()) - 1
    def getYtrainAndXtrain():
        dfTrain = pd.read_csv(CriticTrainingFunctions.stateFile)
        yTrain = dfTrain.pop("critic")
        dfTrain.pop("won")
        xTensor = tf.convert_to_tensor(dfTrain)
        xTensor = tf.reshape(xTensor, [CriticTrainingFunctions.getFileSize(), 1, SEQUENTIALINPUTSHAPE[0]])
        yTensor = tf.convert_to_tensor(yTrain)
        return xTensor, yTensor
    def trainModel(model):
        xTrain, yTrain = CriticTrainingFunctions.getYtrainAndXtrain()
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss = 'mse', metrics = [tf.keras.metrics.MeanSquaredError()])
        model.fit(xTrain, yTrain, epochs = 20)
        print(model(xTrain))
        print(yTrain)

12
Epoch 1/20
366/366 [==============================] - 1s 852us/step - loss: 0.1549 - mean_squared_error: 0.1549
Epoch 2/20
366/366 [==============================] - 0s 853us/step - loss: 0.0892 - mean_squared_error: 0.0892
Epoch 3/20
366/366 [==============================] - 0s 888us/step - loss: 0.0882 - mean_squared_error: 0.0882
Epoch 4/20
366/366 [==============================] - 0s 896us/step - loss: 0.0882 - mean_squared_error: 0.0882
Epoch 5/20
366/366 [==============================] - 0s 805us/step - loss: 0.0879 - mean_squared_error: 0.0879
Epoch 6/20
366/366 [==============================] - 0s 878us/step - loss: 0.0878 - mean_squared_error: 0.0878
Epoch 7/20
366/366 [==============================] - 0s 916us/step - loss: 0.0872 - mean_squared_error: 0.0872
Epoch 8/20
366/366 [==============================] - 0s 839us/step - loss: 0.0870 - mean_squared_error: 0.0870
Epoch 9/20
366/366 [==============================] - 0s 847us/step - loss: 0.0869 - mean_squared_err